# FOO SPY

Playground for stock ticker computations

* Yahoo Finanace as data source
* SPY, AAPL, etc.
* Adjusted Close (takes in account splits and dividends)
* ILS currency adjusted

In [1]:
def is_module_loaded(module):
    import sys
    return module in sys.modules

In [2]:
#if not is_module_loaded('pandas_datareader'):
#    # https://github.com/pydata/pandas-datareader/issues/958
#    import sys
#    !{sys.executable} -m pip install git+https://github.com/raphi6/pandas-datareader@22f1b951fdd49f30a5832cbca0119f5a2e6da15f
#    import pandas_datareader as dtr

if not is_module_loaded('pymaya'):
    !{sys.executable} -m pip install pymaya

#if not is_module_loaded('yfinance'):
#    !{sys.executable} -m pip install yfinance

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [6]:

import process
import importlib
importlib.reload(process)


<module 'process' from '/Users/ii/Documents/pyfinance500/process.py'>

In [7]:
from datetime import datetime, timedelta

#startdate = datetime(2020, 3, 15)
#enddate = datetime(2023, 1, 1)

startdate = datetime.now() - timedelta(days=365)*5
enddate = datetime.today()


ds = process.yahoo()

portfolio = ['SPY', 'AAPL', 'AVGO', 'IS-FF702.TA']
portfolio = ['SPY', 'AAPL', 'AVGO', 'VTI', 'vt', 'VXUS', 'BND', 'ACWI', 'gld']
portfolio_ils = ['TA35.TA']

data = ds.get_ticker(portfolio, start=startdate, end=enddate)
df_ils = ds.get_ticker(['ILS=X',], start=startdate, end=enddate)
data_ils = ds.get_ticker(portfolio_ils, start=startdate, end=enddate)


In [ ]:
data.head()

,AAPL,ACWI,AVGO,BND,GLD,SPY,TXN,VT,VTI,VXUS
Date,,,,,,,,,,
2018-02-15,41.228756,67.033234,208.443726,69.931694,128.380005,250.677887,91.365242,67.946381,128.651566,50.008968
2018-02-16,41.095287,67.105888,206.010239,70.081276,127.959999,250.751312,91.277885,68.000206,128.706741,50.078041
2018-02-20,40.957066,66.542664,206.614471,70.010910,126.239998,249.181274,91.994339,67.488861,127.935059,49.534092
2018-02-21,40.771160,66.251976,205.786774,69.746925,125.660004,247.941849,90.570145,67.237663,127.337982,49.413212
2018-02-22,41.111961,66.388245,206.391006,69.834908,126.300003,248.263199,90.806068,67.354286,127.420624,49.637695


In [ ]:
# Some stats
data.isnull().sum()
data.agg(['min', 'max', 'count'])

,AAPL,ACWI,AVGO,BND,GLD,SPY,TXN,VT,VTI,VXUS
min,34.257282,51.647739,151.049515,68.736015,111.099998,213.785492,78.270370,51.483624,106.781784,33.816257
max,180.683853,104.551758,653.269958,84.423752,193.889999,470.083710,193.179932,106.077019,239.077789,63.633888
count,1257.000000,1257.000000,1257.000000,1257.000000,1257.000000,1257.000000,1257.000000,1257.000000,1257.000000,1257.000000


In [ ]:
# Adjust currency
df_ils = df_ils["ILS=X"]
q = data.multiply(df_ils, axis="index")

# Add ILS ticker column
q = q.join(data_ils)
q = q.join(df_ils)

# Fix non-trading dates
q = q.fillna(method="ffill")
q = q.fillna(method="bfill")


In [ ]:
q

,AAPL,ACWI,AVGO,BND,GLD,SPY,TXN,VT,VTI,VXUS,TA35.TA,ILS=X
Date,,,,,,,,,,,,
2018-02-14,145.160676,236.014628,733.901167,246.219701,452.008018,882.601734,321.684222,239.229691,452.964146,176.074574,1474.780029,3.53493
2018-02-15,145.160676,236.014628,733.901167,246.219701,452.008018,882.601734,321.684222,239.229691,452.964146,176.074574,1490.199951,3.52086
2018-02-16,145.099656,236.938150,727.383035,247.443678,451.802459,885.355270,322.284881,240.095815,454.439065,176.816054,1490.199951,3.53081
2018-02-19,145.099656,236.938150,727.383035,247.443678,451.802459,885.355270,322.284881,240.095815,454.439065,176.816054,1508.099976,3.54096
2018-02-20,143.552879,233.229378,724.175470,245.385443,442.466151,873.370415,322.436484,236.545763,448.407271,173.615014,1498.849976,3.50496
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-07,536.405691,319.833682,2134.482571,254.803955,604.373442,1442.291165,637.409408,325.322308,725.609441,193.595427,1803.280029,3.47381
2023-02-08,527.576802,317.474571,2092.765487,255.565654,606.603219,1428.252984,613.872289,323.039414,718.734871,192.961101,1794.890015,3.47803
2023-02-09,525.341942,316.934907,2090.382373,255.207937,603.424827,1419.685676,610.260155,322.305507,713.661545,193.829701,1782.949951,3.48740


In [ ]:
m = process.maya()
#s1 =  m.get_ticker("1159250", startdate, enddate)
#q = q.join(s1)

In [ ]:
# Now we can calculate returns

cum_return = (q.iloc[-1] / q.iloc[0]) - 1
cum_return
# Get as percentage
cum_return * 100

AAPL       272.386200
ACWI        36.536088
AVGO       187.069681
BND          4.179544
GLD         33.954491
SPY         64.202628
TXN         93.044070
VT          36.932826
VTI         60.768990
VXUS        11.301900
TA35.TA     22.218902
ILS=X       -0.637356
dtype: float64

In [ ]:
# compute daily returns using pandas pct_change()
df_daily_returns = q.pct_change()
# skip first row with NA 
df_daily_returns = df_daily_returns[1:]
df_daily_returns

,AAPL,ACWI,AVGO,BND,GLD,SPY,TXN,VT,VTI,VXUS,TA35.TA,ILS=X
Date,,,,,,,,,,,,
2018-02-15,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010456,-0.003980
2018-02-16,-0.000420,0.003913,-0.008881,0.004971,-0.000455,0.003120,0.001867,0.003620,0.003256,0.004211,0.000000,0.002826
2018-02-19,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012012,0.002875
2018-02-20,-0.010660,-0.015653,-0.004410,-0.008318,-0.020665,-0.013537,0.000470,-0.014786,-0.013273,-0.018104,-0.006134,-0.010167
2018-02-21,-0.006612,-0.006442,-0.006080,-0.005846,-0.006668,-0.007046,-0.017532,-0.005797,-0.006740,-0.004518,0.002268,-0.002083
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-07,0.036251,0.027510,0.038919,0.015301,0.017621,0.029982,0.030672,0.027548,0.029500,0.023665,-0.005685,0.016685
2023-02-08,-0.016459,-0.007376,-0.019544,0.002989,0.003689,-0.009733,-0.036926,-0.007017,-0.009474,-0.003277,-0.004653,0.001215
2023-02-09,-0.004236,-0.001700,-0.001139,-0.001400,-0.005240,-0.005998,-0.005884,-0.002272,-0.007059,0.004501,-0.006652,0.002694


In [ ]:
# Calculate the cumulative daily returns
df_cum_daily_returns = (1 + df_daily_returns).cumprod() - 1
df_cum_daily_returns = df_cum_daily_returns.reset_index()
df_cum_daily_returns

,Date,AAPL,ACWI,AVGO,BND,GLD,SPY,TXN,VT,VTI,VXUS,TA35.TA,ILS=X
0,2018-02-15,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010456,-0.003980
1,2018-02-16,-0.000420,0.003913,-0.008881,0.004971,-0.000455,0.003120,0.001867,0.003620,0.003256,0.004211,0.010456,-0.001165
2,2018-02-19,-0.000420,0.003913,-0.008881,0.004971,-0.000455,0.003120,0.001867,0.003620,0.003256,0.004211,0.022593,0.001706
3,2018-02-20,-0.011076,-0.011801,-0.013252,-0.003388,-0.021110,-0.010459,0.002339,-0.011219,-0.010060,-0.013969,0.016321,-0.008478
4,2018-02-21,-0.017615,-0.018167,-0.019252,-0.009214,-0.027637,-0.017432,-0.015234,-0.016951,-0.016732,-0.018424,0.018626,-0.010543
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1298,2023-02-07,2.695255,0.355143,1.908406,0.034864,0.337086,0.634136,0.981476,0.359874,0.601914,0.099508,0.222745,-0.017290
1299,2023-02-08,2.634433,0.345148,1.851563,0.037958,0.342019,0.618230,0.908307,0.350332,0.586737,0.095906,0.217056,-0.016097
1300,2023-02-09,2.619038,0.342861,1.848316,0.036505,0.334987,0.608524,0.897078,0.347264,0.575537,0.100839,0.208960,-0.013446
1301,2023-02-10,2.634813,0.344226,1.824396,0.034073,0.340072,0.615338,0.909369,0.347338,0.581381,0.097769,0.208960,-0.011573


In [ ]:
# Normalize data
def normalize_data(df):
    return df / df.iloc[0, :]


In [ ]:
# Cummulative returns (in percent) at end of period
cum_return_entire_period = df_cum_daily_returns.iloc[:, 1:].tail(1)
cum_return_entire_period * 100

,AAPL,ACWI,AVGO,BND,GLD,SPY,TXN,VT,VTI,VXUS,TA35.TA,ILS=X
1302,272.3862,36.536088,187.069681,4.179544,33.954491,64.202628,93.04407,36.932826,60.76899,11.3019,22.218902,-0.637356


In [ ]:
df2 = df_cum_daily_returns.melt(id_vars=['Date'], var_name='Symbols', value_name='cum_return')
df2['cum_return_pct'] = df2['cum_return'] * 100
df2


,Date,Symbols,cum_return,cum_return_pct
0,2018-02-15,AAPL,0.000000,0.000000
1,2018-02-16,AAPL,-0.000420,-0.042036
2,2018-02-19,AAPL,-0.000420,-0.042036
3,2018-02-20,AAPL,-0.011076,-1.107598
4,2018-02-21,AAPL,-0.017615,-1.761512
...,...,...,...,...
15631,2023-02-07,ILS=X,-0.017290,-1.729031
15632,2023-02-08,ILS=X,-0.016097,-1.609651
15633,2023-02-09,ILS=X,-0.013446,-1.344579
15634,2023-02-10,ILS=X,-0.011573,-1.157307


In [ ]:
if not is_module_loaded('plotly'):
    !{sys.executable} -m pip install plotly

    import plotly.offline as offline
    from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
    import plotly.express as px
    import plotly.graph_objs as go
    offline.init_notebook_mode(connected=True)

In [ ]:
fig = px.line(df2, x='Date',
              y='cum_return_pct', color='Symbols',
              title='Performance - Daily Cumulative Returns',
              labels={'cum_return_pct':'daily cumulative returns (%)', })
fig.show()
